# Applying Heuristics

It is often the case for geometry problems fed to Newclid that they cannot be solved right away with the initial information. The way to expand its reasoning, from the introduction of the JGEX system, was to mimic the human strategy of "adding an auxiliary point" to the problem, in order to be able to justify new logical developments. The most successful way to date to generate those new points is to do it with the help of a trained language model capable of processing JGEX problems and proposing new points in case Newclid can't solve them.

At the start of JGEX though, language models were not available. And even today they are very costly and not as easily accessible as Newclid itself. This creates an incentive to the development of human heuristics for the suggestions of auxiliary points, that is, scripted ways to suggest those new points and try to solve problems without having to call for the help of large language models.

In this tutorial we show how to test the heuristics that are built in Newclid.

## ProblemSetup: a list of problems

We will focus on how to apply a list of heuristics to add points to a collection of problems encoded as JGEX problems on a file, and then collect statistics on the success.

First, we turn our list of problems into a dictionary that can be manipulated.

In [1]:
from pathlib import Path

from newclid.jgex.formulation import jgex_formulation_from_txt_file

DATASET_PATH = Path("../newclid/problems_datasets")
problem_file_name = "jgex_ag_231.txt"

EVAL_PROBLEMS = jgex_formulation_from_txt_file(DATASET_PATH.joinpath(problem_file_name))

## Heuristics list

We currently have 8 kinds of heuristics implemented. For an easier selection of heuristics to try, we leave them as a dictionary with string keys.

In [2]:
from newclid.heuristics.angle_vertices import AngleVerticesHeuristicConfig
from newclid.heuristics.centers_of_cyclic import CentersOfCyclicHeuristicConfig
from newclid.heuristics.circumcircles_for_eqangle import (
    ThreeCircumcirclesForEqangleGoalHeuristicConfig,
)
from newclid.heuristics.foots_on_lines import FootHeuristicConfig
from newclid.heuristics.line_intersections import LineIntersectionsHeuristicConfig
from newclid.heuristics.reflect_on_center import ReflectOnCenterHeuristicConfig
from newclid.heuristics.transfer_distances import TransferDistancesHeuristicConfig

ALL_HEURISTICS = {
    "angle_vertices": AngleVerticesHeuristicConfig(),
    "line_intersections": LineIntersectionsHeuristicConfig(),
    "centers_of_cyclic": CentersOfCyclicHeuristicConfig(),
    "reflect_on_center": ReflectOnCenterHeuristicConfig(),
    "midpoint": ReflectOnCenterHeuristicConfig(),
    "compasses": TransferDistancesHeuristicConfig(),
    "foot_on_lines": FootHeuristicConfig(),
    "three_circumcircles": ThreeCircumcirclesForEqangleGoalHeuristicConfig(),
}

## Combine a list of heuristiscs into a function

Next, we create a function that will apply a list of heuristics given as a list of strings.

In [3]:
import logging

import numpy
from newclid.heuristics.apply_heuristics import apply_heuristics_on_nc_problem
from newclid.heuristics.heuristic_from_config import (
    HeuristicConfig,
    heuristic_from_config,
)
from newclid.jgex.formulation import JGEXFormulation
from newclid.predicate_types import PredicateArgument
from newclid.problem import ProblemSetup

LOGGER = logging.getLogger(__name__)


def apply_selected_heuristics(
    problem_setup: ProblemSetup,
    jgex_problem: JGEXFormulation,
    selected_heuristics: list[str],
    rng: numpy.random.Generator,
    max_new_points: int | None = None,
) -> ProblemSetup:
    heuristics_config: list[HeuristicConfig] = []
    for heuristic_name in selected_heuristics:
        if heuristic_name not in ALL_HEURISTICS:
            LOGGER.info(
                f"Heuristic '{heuristic_name}' is not defined in ALL_HEURISTICS. Skipping it."
            )
        else:
            heuristics_config.append(ALL_HEURISTICS[heuristic_name])

    if not heuristics_config:
        LOGGER.warning(
            "No valid heuristics were selected. Returning the original problem."
        )
        return problem_setup

    new_problem = problem_setup.model_copy(deep=True)
    new_points: set[PredicateArgument] = set()
    max_new_points = max_new_points if max_new_points is not None else 100
    for heuristic_config in heuristics_config:
        heuristic = heuristic_from_config(heuristic_config)
        problem_setup, added_clauses_consequences = apply_heuristics_on_nc_problem(
            problem_setup=problem_setup,
            jgex_problem=jgex_problem,
            heuristic=heuristic,
            rng=rng,
            max_new_points=max_new_points - len(new_points),
        )
        new_problem = problem_setup
        for clause_consequences in added_clauses_consequences:
            new_points.update(clause_consequences.new_points.keys())

    return new_problem

## Choosing heuristics

With this preparation, we can easily choose the list of heuristics we will try to apply using simply the string names we established in the list of `ALL_HEURISTICS` before (change this list to try different combinations). You should also choose the maximum number of extra points you are willing to try.

In [4]:
heuristics_to_apply = ["midpoint", "compasses", "foot_on_lines", "three_circumcircles"]
max_points = 50

## Applying heuristics and getting statistics

We can finally apply the heuristics to our list of problems and gather statistics.

In [ ]:
from newclid.api import GeometricSolverBuilder
from newclid.jgex.problem_builder import JGEXProblemBuilder

logging.basicConfig(level=logging.CRITICAL)

rng = numpy.random.default_rng(0)

built_problems = 0
not_built_problems = 0
solved_before_heuristics = 0
solved_with_heuristics = 0
not_solved_with_heuristics = 0

for problem_name, problem in EVAL_PROBLEMS.items():
    description_string = problem_name + ": "
    try:
        problem_builder = (
            JGEXProblemBuilder(rng)
            .include_auxiliary_clauses(True)
            .with_problem(problem)
        )
        problem_setup = problem_builder.build()
        built_problems += 1
        description_string += "Built | "
    except Exception:
        not_built_problems += 1
        description_string += "Not built"
        print(description_string)
        continue
    solver = GeometricSolverBuilder(rng).build(problem_setup)
    success = solver.run()
    if success:
        solved_before_heuristics += 1
        description_string += "Solved without heuristics"
        print(description_string)
        continue
    else:
        new_problem = apply_selected_heuristics(
            problem_setup, problem, heuristics_to_apply, rng, max_points
        )
        new_solver = GeometricSolverBuilder().build(new_problem)
        new_success = new_solver.run()

        if new_success:
            solved_with_heuristics += 1
            description_string += f"Solved with heuristics {heuristics_to_apply}."
        else:
            not_solved_with_heuristics += 1
            description_string += f"Not solved with heuristics {heuristics_to_apply}."

        print(description_string)

print("\nEvaluation results:\n")
print(
    f"Problems solved before heuristics: {solved_before_heuristics}/{len(EVAL_PROBLEMS)}"
)
print(
    f"Problems solved with heuristics {heuristics_to_apply}: {solved_with_heuristics}/{len(EVAL_PROBLEMS)}"
)
print(
    f"Problems not solved with heuristics: {not_solved_with_heuristics}/ {len(EVAL_PROBLEMS)}"
)

examples/complete2/012/complete_004_6_GDD_FULL_81-109_101.gex: Built | Solved without heuristics
examples/complete2/012/complete_002_6_GDD_FULL_41-60_59.gex: Built | Solved without heuristics
examples/complete2/012/complete_002_6_GDD_FULL_01-20_04.gex: Built | Solved without heuristics
examples/complete2/012/complete_004_6_GDD_FULL_81-109_90.gex: Built | Solved without heuristics
examples/complete2/012/complete_004_6_GDD_FULL_81-109_94.gex: Built | Solved without heuristics
examples/complete2/012/complete_003_6_GDD_FULL_21-40_37.gex: Built | Solved without heuristics
examples/complete2/012/complete_003_6_GDD_FULL_21-40_22.gex: Built | Solved without heuristics
examples/complete2/012/complete_001_6_GDD_FULL_01-20_19.gex: Built | Solved without heuristics
examples/complete2/012/complete_001_6_GDD_FULL_61-80_74.gex: Built | Solved without heuristics
examples/complete2/013/complete_002_6_GDD_FULL_41-60_49.gex: Built | Solved without heuristics
examples/complete2/013/complete_006_Other_ndgT